## YouTube Extractor
- YouTubeのURLを入力するとその動画のテキストを抽出し検索できる
- GPUを使用しないと動画からテキストを抽出する時間がかなり遅くなるのでGPUの利用を推奨

セットアップ

In [ ]:
!pip install langchain
!pip install openai
!pip install langchain-openai
!pip install pytube
!pip install git+https://github.com/openai/whisper
!pip install faiss-gpu
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.8/811.8 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.8/394.8 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 44.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

google colab用

In [ ]:
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

YouTubeの動画からテキストとそのテキストの開始時間を抽出

In [ ]:
import whisper
import pytube

url = "https://www.youtube.com/watch?v=gcOlzvwxVw8"
video = pytube.YouTube(url)
audio = video.streams.get_audio_only()
path = audio.download(output_path="audio.mp3")
model = whisper.load_model("medium")
transcription = model.transcribe(path)
transcription["segments"][0]

テキストとそのテキストの開始時間を配列に変換

In [ ]:
from datetime import datetime
texts = []
start_times = []

for segment in transcription['segments']:
    text = segment['text']
    start = segment['start']
    
    start_datetime = datetime.fromtimestamp(start).strftime('%H:%M:%S')
    texts.append("".join(text))
    start_times.append(start_datetime)

print(texts[:10])
print(start_times[:10])

テキストを一定のチャンクに分割
ベクターストアを作成

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
# UserWarning: `VectorDBQAWithSourcesChain` is deprecated - please use `from langchain.chains import RetrievalQAWithSourcesChain`
from langchain.chains import VectorDBQAWithSourcesChain
from langchain_openai import OpenAI
import openai
import faiss

docs = []
metadatas = []
text_splitter = CharacterTextSplitter(chunk_size=1500, separator="\n")
for i, d in enumerate(texts):
  splits = text_splitter.split_text(d)
  docs.extend(splits)
  metadatas.extend([{"source": start_times[i]}]*len(splits))

embeddings = OpenAIEmbeddings()
store = FAISS.from_texts(docs, embeddings, metadatas=metadatas)
faiss.write_index(store.index, "docs.index")

chain = VectorDBQAWithSourcesChain.from_llm(llm=OpenAI(), vectorstore=store)

実行

In [ ]:
result = chain.invoke({"question": "ジェミニウルトラはGPT4と比較して何が優れていますか？"})
print(result['answer'])
print(result['sources'])

for source in result['sources'].replace(' ', '').split(","):
  print(texts[start_times.index(source)])